In [16]:
import pandas as pd
import numpy as np

In [17]:
df = pd.read_csv("processed.csv")
df

,Unnamed: 0,clean_comment,category
0,0,family mormon never tried explain still stare ...,1
1,1,buddhism much lot compatible christianity espe...,1
2,2,seriously say thing first get complex explain ...,-1
3,3,learned want teach different focus goal not wr...,0
4,4,benefit may want read living buddha living chr...,1
...,...,...,...
36241,37243,agree push make nation either pity pakistan in...,-1
36242,37244,jesus,0
36243,37245,kya bhai pure saal chutiya banaya modi aur jab...,1
36244,37246,downvote karna tha par upvote hogaya,0


In [18]:
df.dropna(inplace=True)

In [19]:
import dagshub
dagshub.init(repo_owner='rajatchauhan99', repo_name='Youtube-Comment-Analysis-Chrome-Plugin', mlflow=True)

Initialized MLflow to track repo "rajatchauhan99/Youtube-Comment-Analysis-Chrome-Plugin"

Repository rajatchauhan99/Youtube-Comment-Analysis-Chrome-Plugin initialized!

In [20]:
# Set or create an experiment
import mlflow
mlflow.set_experiment("Exp 5 - Trying Different ML Algos with HP Tuning")

<Experiment: artifact_location='mlflow-artifacts:/96c047f2f48c4d168382939725e408d8', creation_time=1729342817379, experiment_id='5', last_update_time=1729342817379, lifecycle_stage='active', name='Exp 5 - Trying Different ML Algos with HP Tuning', tags={}>

In [21]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [22]:
# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

ngram_range = (1, 3)  # Trigram setting
max_features = 1000  # Set max_features to 1000 for TF-IDF

# Step 4: Train-test split before vectorization and resampling
X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

# Step 2: Vectorization using TF-IDF, fit on training data only
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)  # Fit on training data
X_test_vec = vectorizer.transform(X_test)  # Transform test data

smote = SMOTE(random_state=42)
X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for XGBoost
def objective_xgboost(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42)
    return accuracy_score(y_test, model.fit(X_train_vec, y_train).predict(X_test_vec))


# Step 7: Run Optuna for XGBoost, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = XGBClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42)

    # Log the best model with MLflow, passing the algo_name as "xgboost"
    log_mlflow("XGBoost", best_model, X_train_vec, X_test_vec, y_train, y_test)

# Run the experiment for XGBoost
run_optuna_experiment()


[I 2024-10-19 18:30:39,977] A new study created in memory with name: no-name-237ed102-2aa1-445c-90e9-634c2d47b497
[I 2024-10-19 18:31:15,119] Trial 0 finished with value: 0.6582976962339633 and parameters: {'n_estimators': 169, 'learning_rate': 0.012435543967852432, 'max_depth': 7}. Best is trial 0 with value: 0.6582976962339633.
[I 2024-10-19 18:31:47,267] Trial 1 finished with value: 0.6599530969788936 and parameters: {'n_estimators': 203, 'learning_rate': 0.01233988167940562, 'max_depth': 6}. Best is trial 1 with value: 0.6599530969788936.
[I 2024-10-19 18:33:07,013] Trial 2 finished with value: 0.7846599530969789 and parameters: {'n_estimators': 269, 'learning_rate': 0.061278889002356694, 'max_depth': 9}. Best is trial 2 with value: 0.7846599530969789.
[I 2024-10-19 18:34:40,785] Trial 3 finished with value: 0.5632501034625466 and parameters: {'n_estimators': 283, 'learning_rate': 0.0010712481173388547, 'max_depth': 6}. Best is trial 2 with value: 0.7846599530969789.
[I 2024-10-19 